# Expand game categorical values (developer, genres, tags)

In [3]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import urllib
from urllib.request import Request, urlopen
import json

In [4]:
# import all scraped/cleaned data
users_games = pd.read_csv('user_games.txt', header=None, names=['game_id','play_time','user_id'], delimiter=' ')
reviews = pd.read_csv('reviews_cleaned.csv', index_col=0)
#games = pd.read_csv('games_cleaned.csv', index_col=0)
dlcs = pd.read_csv('dlcs_cleaned.csv', index_col=0)

In [9]:
len(reviews['final_user_id'].unique())

57623

#### expand game vars

In [3]:
games.drop(games.loc[pd.isnull(games['developer'])].index, inplace=True)
games = games.reset_index()

In [7]:
def col_to_list(row):
    """This function takes a column and converts the str content to a list"""
    import re
    import html
    
    # strings to replace
    rep = {"'": "", "' ": "", " '": "", '" ' : "", ' "': "", ' ': ''}

    # escape special chars
    rep = dict((re.escape(k), v) for k, v in rep.items())
    
    # compile regex pattern
    pattern = re.compile("|".join(rep.keys()))
    
    # find pattern and replace
    text = html.unescape(pattern.sub(lambda m: rep[re.escape(m.group(0))], row['specs'].strip("[]")))

    return text.split(',')

lst = games.apply(col_to_list, axis=1)

In [8]:
# expand categorical variables
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

X = mlb.fit_transform(lst)
print(X.shape, mlb.classes_)

(17029, 39) ['Captionsavailable' 'Co-op' 'Commentaryavailable'
 'Cross-PlatformMultiplayer' 'DownloadableContent' 'Fullcontrollersupport'
 'Gamedemo' 'Gamepad' 'HTCVive' 'In-AppPurchases' 'IncludesSourceSDK'
 'Includesleveleditor' 'Keyboard/Mouse' 'LocalCo-op' 'LocalMulti-Player'
 'MMO' 'Mods' 'Mods(requireHL2)' 'Multi-player' 'OculusRift' 'OnlineCo-op'
 'OnlineMulti-Player' 'PartialControllerSupport' 'Room-Scale' 'Seated'
 'Shared/SplitScreen' 'Single-player' 'Standing' 'Stats'
 'SteamAchievements' 'SteamCloud' 'SteamLeaderboards' 'SteamTradingCards'
 'SteamTurnNotifications' 'SteamVRCollectibles' 'SteamWorkshop'
 'TrackedMotionControllers' 'ValveAnti-Cheatenabled' 'WindowsMixedReality']


In [9]:
# genres = pd.DataFrame(X, columns=mlb.classes_)
specs = pd.DataFrame(X, columns=mlb.classes_)
# developers = pd.DataFrame(X, columns=mlb.classes_)

In [ ]:
# check for missing data
for column in developers.columns:
    if np.any(pd.isnull(developers[column])) == True:
        print(column)

In [10]:
test = pd.concat([games,genres, specs], axis=1)

In [12]:
test.drop(columns='index', inplace=True)
test

,description,developer,genres,id,price,publisher,release_date,specs,tags,title,...,SteamAchievements,SteamCloud,SteamLeaderboards,SteamTradingCards,SteamTurnNotifications,SteamVRCollectibles,SteamWorkshop,TrackedMotionControllers,ValveAnti-Cheatenabled,WindowsMixedReality
0,"['', 'About This Game', 'Do you love puzzles? ...",ChillFun,['Strategy'],773510,14.99,ChillFun,2018-01-17,"['Single-player', 'Partial Controller Support'...","['Strategy', 'Cartoon', 'Puzzle', 'First-Perso...",The Hardest Thing,...,0,1,1,0,0,0,0,0,0,0
1,"['', 'About This Game', 'In Global Soccer Mana...","gsmpcgame,globalsoccermanager,Andrea Hochstein","['Casual', 'Indie', 'Simulation', 'Sports', 'S...",625700,9.99,gsmpcgame,2017-05-24,"['Single-player', 'Steam Achievements', 'Steam...","['Sports', 'Strategy', 'Simulation', 'Indie', ...",Global Soccer Manager 2017,...,1,0,0,1,0,0,0,0,0,0
2,"['', 'About This Game', 'Experience the purene...",Media Art,"['Adventure', 'Casual']",770880,9.99,Big Fish Games,2018-01-17,['Single-player'],"['Adventure', 'Casual']",Love Story: The Beach Cottage,...,0,0,0,0,0,0,0,0,0,0
3,"['', 'About This Game', 'See how a game create...",AIREM,"['Adventure', 'Indie']",725780,9.99,IQ Publishing,2018-01-17,"['Single-player', 'Full controller support']","['Adventure', 'Indie', 'Horror']",PLAY WITH ME,...,0,0,0,0,0,0,0,0,0,0
4,"['', 'About This Game', 'From the creators of ...",Cellar Door Games,"['Action', 'Adventure', 'Indie', 'RPG']",416600,19.99,Cellar Door Games,2018-01-17,"['Single-player', 'Co-op', 'Online Co-op', 'Lo...","['Indie', 'Action', 'Adventure', 'RPG', 'Co-op...",Full Metal Furies,...,1,1,0,1,0,0,0,0,0,0
5,"['', 'About This Game', '', 'An unсommon puzzl...",OCP,"['Adventure', 'Casual', 'Indie', 'Early Access']",766740,9.99,OCP,2018-01-17,['Single-player'],"['Early Access', 'Casual', 'Indie', 'Adventure...",Maze Of Adventures,...,0,0,0,0,0,0,0,0,0,0
6,"['', 'About This Game', 'Mission Christmas is ...",IndieLip,"['Adventure', 'Casual', 'Indie', 'Simulation']",778450,0.99,IndieLip,2018-01-17,"['Single-player', 'Steam Trading Cards']","['Adventure', 'Indie', 'Casual', 'Simulation']",Christmas Mission,...,0,0,0,1,0,0,0,0,0,0
7,"['', 'About This Game', 'BAE 2 is a simple puz...",Riviysky,"['Casual', 'Indie']",769330,1.99,Riviysky,2018-01-17,"['Single-player', 'Steam Trading Cards']","['Indie', 'Casual']",BAE 2,...,0,0,0,1,0,0,0,0,0,0
8,"['', 'Reviews', '', '“The general feeling is g...",Anamik Majumdar,"['Action', 'Indie']",707320,2.99,Anamik Majumdar,2018-01-17,"['Single-player', 'Steam Achievements', 'Parti...","['Indie', 'Action', 'Platformer']",Keatz: The Lonely Bird,...,1,0,0,0,0,0,0,0,0,0
9,"['', 'About This Game', '4 for the Money is an...",Lost Marble,"['Action', 'Free to Play', 'Indie']",772160,0.00,Lost Marble,2018-01-17,"['Single-player', 'Multi-player', 'Local Multi...","['Action', 'Free to Play', 'Indie']",4 for the Money Demo,...,0,0,0,0,0,0,0,0,0,0


In [73]:
test.to_csv('games_expanded.csv')

In [14]:
games = test

#### expand dlc vars

In [49]:
dlcs.head()
dlcs = dlcs.reset_index()

In [50]:
dlcs

,index,description,developer,genres,id,price,publisher,release_date,specs,tags,title,parent_game_id
0,0,"['', 'About This Content', '►', 'Murum Charta,...",ImaginationOverflow,"['Action', 'Casual', 'Indie']",777450,0.00,NaN,2017-12-30,"['Single-player', 'Downloadable Content', 'Ste...","['Action', 'Indie', 'Casual']",Stellar Interface - Murum Charta,517330
1,1,"['', 'About This Content', 'Unlocks three new ...",Stegalosaurus Game Development,"['Adventure', 'Indie', 'RPG']",694780,0.99,NaN,2018-01-11,"['Single-player', 'Downloadable Content', 'Ste...","['Adventure', 'RPG', 'Indie']","SUPER ARMY OF TENTACLES 3, Winter Outfit Pack ...",592200
2,2,"['', 'About This Content', 'The Wilmington Int...",Drawbridge Designs,['Simulation'],623613,19.99,Aerosoft GmbH,2018-01-11,"['Single-player', 'Online Multi-Player', 'Loca...",['Simulation'],X-Plane 11 - Add-on: Aerosoft - Airport Wilmin...,269950
3,3,"['', 'About This Content', 'Get ready for anot...",Thomson Interactive,['Simulation'],642803,19.99,Dovetail Games - Trains,2018-01-11,"['Single-player', 'Downloadable Content', 'Ste...",['Simulation'],Train Simulator: RhB Enhancement Pack 02 Add-On,24010
4,4,"['', 'About This Content', 'With its three dar...",Bryan Minus,"['Adventure', 'Casual', 'Indie']",722550,1.99,Minus Equals Plus,2018-01-11,"['Single-player', 'Downloadable Content']","['Adventure', 'Indie', 'Casual']",Waiting For the Loop Official Soundtrack and EP,717830
5,5,"['', 'About This Content', 'Banyu Lintar Angin...",Mojiken Studio,"['Casual', 'Indie']",760610,2.99,Toge Productions,2017-03-03,"['Single-player', 'Downloadable Content']","['Indie', 'Casual']",Banyu Lintar Angin - Little Storm - Deluxe Edi...,744800
6,6,"['', 'About This Content', 'A Raven Monologue ...",Mojiken Studio,"['Casual', 'Indie']",763280,2.99,Toge Productions,2018-01-11,"['Single-player', 'Downloadable Content']","['Indie', 'Casual']",A Raven Monologue Fan Pack,744810
7,7,"['', 'About This Content', 'Disorderia (Soundt...",JustE A,['Indie'],783750,0.99,JustE Publishing,2018-01-11,"['Single-player', 'Downloadable Content', 'Ste...",['Indie'],Truth: Disorder - Soundtrack,755350
8,8,"['', 'About This Content', 'DLC with full upgr...",Drift Physics Crew,"['Racing', 'Simulation', 'Sports']",756720,2.99,Drift Physics Crew,2018-01-11,"['Single-player', 'Multi-player', 'Online Mult...","['Simulation', 'Racing', 'Sports']",FURIDASHI - PREMIUM CAR: 2015 STRONGER,658570
9,9,"['', 'About This Content', 'DLC with full upgr...",Drift Physics Crew,"['Racing', 'Simulation', 'Sports']",756729,2.99,Drift Physics Crew,2018-01-11,"['Single-player', 'Multi-player', 'Online Mult...","['Simulation', 'Racing', 'Sports']",FURIDASHI - PREMIUM CAR: 1986 AE-86S,658570


In [52]:
dlcs.drop(columns ='index', inplace=True)

In [53]:
def col_to_list(row):
    """This function takes a column and converts the str content to a list"""
    import re
    import html
    
    # strings to replace
    rep = {"'": "", "' ": "", " '": "", '" ' : "", ' "': "", ' ': ''}

    # escape special chars
    rep = dict((re.escape(k), v) for k, v in rep.items())
    
    # compile regex pattern
    pattern = re.compile("|".join(rep.keys()))
    
    # find pattern and replace
    text = html.unescape(pattern.sub(lambda m: rep[re.escape(m.group(0))], row['specs'].strip("[]")))

    return text.split(',')

lst = dlcs.apply(col_to_list, axis=1)

In [54]:
print(len(lst))
print(len(dlcs))

12571
12571


In [55]:
# expand categorical variables
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

X = mlb.fit_transform(lst)
print(X.shape, mlb.classes_)

(12571, 36) ['Captionsavailable' 'Co-op' 'Commentaryavailable'
 'Cross-PlatformMultiplayer' 'DownloadableContent' 'Fullcontrollersupport'
 'Gamepad' 'HTCVive' 'In-AppPurchases' 'IncludesSourceSDK'
 'Includesleveleditor' 'Keyboard/Mouse' 'LocalCo-op' 'LocalMulti-Player'
 'MMO' 'Multi-player' 'OculusRift' 'OnlineCo-op' 'OnlineMulti-Player'
 'PartialControllerSupport' 'Room-Scale' 'Seated' 'Shared/SplitScreen'
 'Single-player' 'Standing' 'Stats' 'SteamAchievements' 'SteamCloud'
 'SteamLeaderboards' 'SteamTradingCards' 'SteamTurnNotifications'
 'SteamVRCollectibles' 'SteamWorkshop' 'TrackedMotionControllers'
 'ValveAnti-Cheatenabled' 'WindowsMixedReality']


In [56]:
specs_dlc = pd.DataFrame(X, columns=mlb.classes_)

In [57]:
test = pd.concat([dlcs, specs_dlc], axis=1)

In [58]:
test

,description,developer,genres,id,price,publisher,release_date,specs,tags,title,...,SteamAchievements,SteamCloud,SteamLeaderboards,SteamTradingCards,SteamTurnNotifications,SteamVRCollectibles,SteamWorkshop,TrackedMotionControllers,ValveAnti-Cheatenabled,WindowsMixedReality
0,"['', 'About This Content', '►', 'Murum Charta,...",ImaginationOverflow,"['Action', 'Casual', 'Indie']",777450,0.00,NaN,2017-12-30,"['Single-player', 'Downloadable Content', 'Ste...","['Action', 'Indie', 'Casual']",Stellar Interface - Murum Charta,...,1,1,0,1,0,0,0,0,0,0
1,"['', 'About This Content', 'Unlocks three new ...",Stegalosaurus Game Development,"['Adventure', 'Indie', 'RPG']",694780,0.99,NaN,2018-01-11,"['Single-player', 'Downloadable Content', 'Ste...","['Adventure', 'RPG', 'Indie']","SUPER ARMY OF TENTACLES 3, Winter Outfit Pack ...",...,1,0,0,0,0,0,0,0,0,0
2,"['', 'About This Content', 'The Wilmington Int...",Drawbridge Designs,['Simulation'],623613,19.99,Aerosoft GmbH,2018-01-11,"['Single-player', 'Online Multi-Player', 'Loca...",['Simulation'],X-Plane 11 - Add-on: Aerosoft - Airport Wilmin...,...,0,0,0,0,0,0,0,0,0,0
3,"['', 'About This Content', 'Get ready for anot...",Thomson Interactive,['Simulation'],642803,19.99,Dovetail Games - Trains,2018-01-11,"['Single-player', 'Downloadable Content', 'Ste...",['Simulation'],Train Simulator: RhB Enhancement Pack 02 Add-On,...,1,0,0,0,0,0,1,0,0,0
4,"['', 'About This Content', 'With its three dar...",Bryan Minus,"['Adventure', 'Casual', 'Indie']",722550,1.99,Minus Equals Plus,2018-01-11,"['Single-player', 'Downloadable Content']","['Adventure', 'Indie', 'Casual']",Waiting For the Loop Official Soundtrack and EP,...,0,0,0,0,0,0,0,0,0,0
5,"['', 'About This Content', 'Banyu Lintar Angin...",Mojiken Studio,"['Casual', 'Indie']",760610,2.99,Toge Productions,2017-03-03,"['Single-player', 'Downloadable Content']","['Indie', 'Casual']",Banyu Lintar Angin - Little Storm - Deluxe Edi...,...,0,0,0,0,0,0,0,0,0,0
6,"['', 'About This Content', 'A Raven Monologue ...",Mojiken Studio,"['Casual', 'Indie']",763280,2.99,Toge Productions,2018-01-11,"['Single-player', 'Downloadable Content']","['Indie', 'Casual']",A Raven Monologue Fan Pack,...,0,0,0,0,0,0,0,0,0,0
7,"['', 'About This Content', 'Disorderia (Soundt...",JustE A,['Indie'],783750,0.99,JustE Publishing,2018-01-11,"['Single-player', 'Downloadable Content', 'Ste...",['Indie'],Truth: Disorder - Soundtrack,...,1,0,0,0,0,0,0,0,0,0
8,"['', 'About This Content', 'DLC with full upgr...",Drift Physics Crew,"['Racing', 'Simulation', 'Sports']",756720,2.99,Drift Physics Crew,2018-01-11,"['Single-player', 'Multi-player', 'Online Mult...","['Simulation', 'Racing', 'Sports']",FURIDASHI - PREMIUM CAR: 2015 STRONGER,...,1,1,0,0,0,0,0,0,0,0
9,"['', 'About This Content', 'DLC with full upgr...",Drift Physics Crew,"['Racing', 'Simulation', 'Sports']",756729,2.99,Drift Physics Crew,2018-01-11,"['Single-player', 'Multi-player', 'Online Mult...","['Simulation', 'Racing', 'Sports']",FURIDASHI - PREMIUM CAR: 1986 AE-86S,...,1,1,0,0,0,0,0,0,0,0


In [59]:
test.to_csv('dlcs_expanded.csv')

In [61]:
dlcs=test

In [31]:
dlcs.columns

Index(['description', 'developer', 'genres', 'id', 'price', 'publisher',
       'release_date', 'specs', 'tags', 'title', 'parent_game_id',
       'Captionsavailable', 'Co-op', 'Commentaryavailable',
       'Cross-PlatformMultiplayer', 'DownloadableContent',
       'Fullcontrollersupport', 'Gamedemo', 'Gamepad', 'HTCVive',
       'In-AppPurchases', 'IncludesSourceSDK', 'Includesleveleditor',
       'Keyboard/Mouse', 'LocalCo-op', 'LocalMulti-Player', 'MMO', 'Mods',
       'Mods(requireHL2)', 'Multi-player', 'OculusRift', 'OnlineCo-op',
       'OnlineMulti-Player', 'PartialControllerSupport', 'Room-Scale',
       'Seated', 'Shared/SplitScreen', 'Single-player', 'Standing', 'Stats',
       'SteamAchievements', 'SteamCloud', 'SteamLeaderboards',
       'SteamTradingCards', 'SteamTurnNotifications', 'SteamVRCollectibles',
       'SteamWorkshop', 'TrackedMotionControllers', 'ValveAnti-Cheatenabled',
       'WindowsMixedReality'],
      dtype='object')

## Store to PSQL
#### Open a connection to the DB and initialize the user table

Some notes: 
- for automatically incrementing PK, use data type `serial`
- for automatically generating a timestamp, use `default(current_timestamp)`
- `current_timestamp` is a psql function that will generate the current time stamp
- the `unique` constraint will allow skipping inserts if duplicate is found

#### populate users table

In [32]:
reviews.head(2)

,compensation,hours,num_reviews,product_id,products,recommended,text,final_user_id,date_iso
0,0,0.0,35,642803,468.0,1,"Okay, get thisyou can open the cab's side door...",76561198067019938,2018-01-13T00:00:00
1,0,0.0,6,760610,888.0,1,Bought for the pure respect of the original pr...,76561197999899413,2018-01-12T00:00:00


In [36]:
# db info
dbname = 'steam_prod'
username = 'hyunjoohwang'

# make a connection to the database
conn = psycopg2.connect(f"dbname={dbname} user={username}")

# open a cursor to perform sql queries
cur = conn.cursor()
print(conn)

# create a table with automatically incrementing PK and automatically generated creation time stamp
sql = """
create table if not exists users (
id serial primary key,
player_id bigint not null unique,
created_at timestamp not null default(current_timestamp),
updated_at timestamp);
"""
cur.execute(sql)
conn.commit()

<connection object at 0x1a13304940; dsn: 'dbname=steam_prod user=hyunjoohwang', closed: 0>


In [37]:
unique_users = reviews.final_user_id.unique()
print('Number of unique users: ' + str(len(unique_users)))

Number of unique users: 57623


In [38]:
# insert users into table users
# current_timestamp is a PSQL function that generates the current time stamp
for user in unique_users:
    sql = f"""
    insert into users (player_id, updated_at) values ({user}, current_timestamp);
    """
    cur.execute(sql)
    conn.commit()

#### populate dlcs table

In [62]:
dlcs.head(50)

,description,developer,genres,id,price,publisher,release_date,specs,tags,title,...,SteamAchievements,SteamCloud,SteamLeaderboards,SteamTradingCards,SteamTurnNotifications,SteamVRCollectibles,SteamWorkshop,TrackedMotionControllers,ValveAnti-Cheatenabled,WindowsMixedReality
0,"['', 'About This Content', '►', 'Murum Charta,...",ImaginationOverflow,"['Action', 'Casual', 'Indie']",777450,0.00,NaN,2017-12-30,"['Single-player', 'Downloadable Content', 'Ste...","['Action', 'Indie', 'Casual']",Stellar Interface - Murum Charta,...,1,1,0,1,0,0,0,0,0,0
1,"['', 'About This Content', 'Unlocks three new ...",Stegalosaurus Game Development,"['Adventure', 'Indie', 'RPG']",694780,0.99,NaN,2018-01-11,"['Single-player', 'Downloadable Content', 'Ste...","['Adventure', 'RPG', 'Indie']","SUPER ARMY OF TENTACLES 3, Winter Outfit Pack ...",...,1,0,0,0,0,0,0,0,0,0
2,"['', 'About This Content', 'The Wilmington Int...",Drawbridge Designs,['Simulation'],623613,19.99,Aerosoft GmbH,2018-01-11,"['Single-player', 'Online Multi-Player', 'Loca...",['Simulation'],X-Plane 11 - Add-on: Aerosoft - Airport Wilmin...,...,0,0,0,0,0,0,0,0,0,0
3,"['', 'About This Content', 'Get ready for anot...",Thomson Interactive,['Simulation'],642803,19.99,Dovetail Games - Trains,2018-01-11,"['Single-player', 'Downloadable Content', 'Ste...",['Simulation'],Train Simulator: RhB Enhancement Pack 02 Add-On,...,1,0,0,0,0,0,1,0,0,0
4,"['', 'About This Content', 'With its three dar...",Bryan Minus,"['Adventure', 'Casual', 'Indie']",722550,1.99,Minus Equals Plus,2018-01-11,"['Single-player', 'Downloadable Content']","['Adventure', 'Indie', 'Casual']",Waiting For the Loop Official Soundtrack and EP,...,0,0,0,0,0,0,0,0,0,0
5,"['', 'About This Content', 'Banyu Lintar Angin...",Mojiken Studio,"['Casual', 'Indie']",760610,2.99,Toge Productions,2017-03-03,"['Single-player', 'Downloadable Content']","['Indie', 'Casual']",Banyu Lintar Angin - Little Storm - Deluxe Edi...,...,0,0,0,0,0,0,0,0,0,0
6,"['', 'About This Content', 'A Raven Monologue ...",Mojiken Studio,"['Casual', 'Indie']",763280,2.99,Toge Productions,2018-01-11,"['Single-player', 'Downloadable Content']","['Indie', 'Casual']",A Raven Monologue Fan Pack,...,0,0,0,0,0,0,0,0,0,0
7,"['', 'About This Content', 'Disorderia (Soundt...",JustE A,['Indie'],783750,0.99,JustE Publishing,2018-01-11,"['Single-player', 'Downloadable Content', 'Ste...",['Indie'],Truth: Disorder - Soundtrack,...,1,0,0,0,0,0,0,0,0,0
8,"['', 'About This Content', 'DLC with full upgr...",Drift Physics Crew,"['Racing', 'Simulation', 'Sports']",756720,2.99,Drift Physics Crew,2018-01-11,"['Single-player', 'Multi-player', 'Online Mult...","['Simulation', 'Racing', 'Sports']",FURIDASHI - PREMIUM CAR: 2015 STRONGER,...,1,1,0,0,0,0,0,0,0,0
9,"['', 'About This Content', 'DLC with full upgr...",Drift Physics Crew,"['Racing', 'Simulation', 'Sports']",756729,2.99,Drift Physics Crew,2018-01-11,"['Single-player', 'Multi-player', 'Online Mult...","['Simulation', 'Racing', 'Sports']",FURIDASHI - PREMIUM CAR: 1986 AE-86S,...,1,1,0,0,0,0,0,0,0,0


In [ ]:
# # create a table with automatically incrementing PK and automatically generated creation time stamp
# sql = """
# create table if not exists dlcs (
# id serial primary key,
# dlc_id int not null unique,
# title text,
# parent_game_id int not null,
# genres text,
# price float,
# developer text,
# publisher text,
# release_date timestamp,
# specs text,
# tags text,
# description text,
# created_at timestamp not null default(current_timestamp),
# updated_at timestamp);
# """
# cur.execute(sql)
# conn.commit()

In [63]:
# prepare df to dump into sql (mainly escape the strings)
df = dlcs

In [ ]:
type(df)
df.columns

In [64]:
# psql doesn't like unescaped symbols so write a function that escapes symbols
def escape_char(text):
    """This function returns the escaped string"""
    import html
    
    try:
        s = html.escape(text)
        return s
    except:
        pass

In [65]:
# escape values
to_escape = ['description', 'genres', 'tags', 'specs', 'title', 'publisher', 'developer']
for col in to_escape:
    df[col] = df[col].apply(lambda x: escape_char(x))

In [66]:
# rename column that psql chokes on
df = df.rename(columns={'Mods(requireHL2)': 'Mods-requireHL2'})

In [67]:
# dump df into table
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
df.to_sql('dlcs', engine, if_exists='append')

#### populate games table

In [70]:
games.head(2)

,description,developer,genres,id,price,publisher,release_date,specs,tags,title,...,SteamAchievements,SteamCloud,SteamLeaderboards,SteamTradingCards,SteamTurnNotifications,SteamVRCollectibles,SteamWorkshop,TrackedMotionControllers,ValveAnti-Cheatenabled,WindowsMixedReality
0,"[&#x27;&#x27;, &#x27;About This Game&#x27;, &#...",ChillFun,[&#x27;Strategy&#x27;],773510,14.99,ChillFun,2018-01-17,"[&#x27;Single-player&#x27;, &#x27;Partial Cont...","[&#x27;Strategy&#x27;, &#x27;Cartoon&#x27;, &#...",The Hardest Thing,...,0,1,1,0,0,0,0,0,0,0
1,"[&#x27;&#x27;, &#x27;About This Game&#x27;, &#...","gsmpcgame,globalsoccermanager,Andrea Hochstein","[&#x27;Casual&#x27;, &#x27;Indie&#x27;, &#x27;...",625700,9.99,gsmpcgame,2017-05-24,"[&#x27;Single-player&#x27;, &#x27;Steam Achiev...","[&#x27;Sports&#x27;, &#x27;Strategy&#x27;, &#x...",Global Soccer Manager 2017,...,1,0,0,1,0,0,0,0,0,0


In [69]:
# repeat above process for games table
df = games
to_escape = ['description', 'genres', 'tags', 'specs', 'title', 'publisher', 'developer']
for col in to_escape:
    df[col] = df[col].apply(lambda x: escape_char(x))

In [71]:
df = df.rename(columns={'Mods(requireHL2)': 'Mods-requireHL2'})

In [72]:
# dump df into table
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
df.to_sql('games', engine, if_exists='append')

#### populate users_games middle table

In [78]:
users_games.head(2)

,game_id,play_time,user_id
0,3320,352,76561197999899413
1,3410,197,76561197999899413


In [79]:
users_games.to_sql('users_games', engine, if_exists='append')

In [ ]:
drop_user = users_games.loc[users_games['user_id']==76561197999899413].index
users_games.drop(drop_user, inplace=True)

In [ ]:
drop_user = users_games.loc[users_games['user_id']==76561198151921721].index
users_games.drop(drop_user, inplace=True)

In [ ]:
#users_games.to_sql('users_games', engine, if_exists='append')

In [ ]:
# map user foreign keys
sql = """
update users_games
set users_fk = users.id
from users
where users_games.user_id = users.player_id;
"""
cur.execute(sql)
conn.commit()

In [ ]:
# map games foreign keys
sql = """
update users_games
set games_fk = games.id
from games
where users_games.game_id = games.game_id;
"""
cur.execute(sql)
conn.commit()

In [ ]:
cur.close()
conn.close()

#### populate reviews table

In [75]:
reviews.head(2)

,compensation,hours,num_reviews,product_id,products,recommended,text,final_user_id,date_iso
0,0,0.0,35,642803,468.0,1,"Okay, get thisyou can open the cab's side door...",76561198067019938,2018-01-13T00:00:00
1,0,0.0,6,760610,888.0,1,Bought for the pure respect of the original pr...,76561197999899413,2018-01-12T00:00:00


In [76]:
df = reviews
to_escape = ['text']
for col in to_escape:
    df[col] = df[col].apply(lambda x: escape_char(x))

In [77]:
df.to_sql('reviews', engine, if_exists='append')

In [ ]:
# make a connection to the database
conn = psycopg2.connect(f"dbname={dbname} user={username}")

# open a cursor to perform sql queries
cur = conn.cursor()

# map foreign keys
sql = """
update reviews
set users_fk = users.id
from users
where reviews.final_user_id = users.player_id;
"""
cur.execute(sql)
conn.commit()